# Partial Exam of Spark

@roman

20 apr 2024

---
# Settings

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1713833005447_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# init spark session
spark = SparkSession.builder.appName('QPP').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# bucket
NAME = 'roman'
BUCKET = f"s3://itam-analytics-{NAME}"
FOLDER = 'qqp'
NAME_FILE = 'qqp'

# type of catalog
CATALOG_TYPE = 'medicamentos'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
# Data

In [4]:
# read raw data from S3
df_qqp = spark.read.csv(f"{BUCKET}/{FOLDER}/{NAME_FILE}.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# look columns
df_qqp.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product: string (nullable = true)
 |-- presentation: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- catalog: string (nullable = true)
 |-- price: double (nullable = true)
 |-- created_at: date (nullable = true)
 |-- store: string (nullable = true)
 |-- type_of_store: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)

In [6]:
# save to parquet in s3, partioned by catalog type
df_qqp.write.mode('overwrite').partitionBy('catalog').parquet(f"{BUCKET}/{FOLDER}/{NAME_FILE}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
# Questions

## S1: General

### Get Data

In [4]:
# read parquet from s3
df_qqp = spark.read.parquet(f"{BUCKET}/{FOLDER}/{NAME_FILE}")

# look columns
df_qqp.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product: string (nullable = true)
 |-- presentation: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- created_at: date (nullable = true)
 |-- store: string (nullable = true)
 |-- type_of_store: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- catalog: string (nullable = true)

### Wrangle

In [5]:
# from the column created_at, extract the year
df_qqp = df_qqp.withColumn('year', F.year('created_at'))

# if catalog is in 's/m', 'sin marca' or 'cualquier marca' then set 's/m'
df_qqp = (
    df_qqp
    .withColumn(
        'catalog', F.when(
                col('catalog').isin('s/m', 'sin marca', 'cualquier marca'), 's/m'
            ).otherwise(
                col('catalog')
            )
        )
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# look # of rows  per year
df_qqp.groupBy('year').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|year|   count|
+----+--------+
|2018|16782901|
|2019|13682091|
|2020|10780481|
|2021|12813994|
|2022|14449481|
|2023|14782812|
|2024| 3624947|
+----+--------+

### Q1: ¿Cuántos catálogos diferentes tenemos?

In [7]:
# num different catalogs
df_qqp.agg(F.countDistinct('catalog')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|count(DISTINCT catalog)|
+-----------------------+
|                     12|
+-----------------------+

In [8]:
# num different catalogs per year
df_qqp.groupBy('year').agg(F.countDistinct('catalog')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------------+
|year|count(DISTINCT catalog)|
+----+-----------------------+
|2018|                     10|
|2019|                     11|
|2020|                     10|
|2021|                     10|
|2022|                     11|
|2023|                     11|
|2024|                     10|
+----+-----------------------+

En total hay 12 catálogos diferentes, sin embargo no todos los catálogos están presentes en todos los años. Por ejemplo, en 2018 solo hay 10 catálogos diferentes.

### Q2: ¿Cuáles son los 20 catálogos con más observaciones?

In [9]:
# count num of catalogs
(
    df_qqp
    .groupBy('catalog')
    .count()
    .orderBy('count', ascending=False)
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|            catalog|   count|
+-------------------+--------+
|            basicos|46967421|
|       medicamentos|19223453|
|  electrodomesticos| 7175538|
| frutas y legumbres| 5041732|
|   utiles escolares| 2936010|
|           mercados| 2238684|
|           juguetes| 1432183|
|              pacic| 1079162|
|pescados y mariscos|  569632|
|          navidenos|  236543|
|              tenis|   15768|
|        aeropuertos|     581|
+-------------------+--------+

Los productos que más se venden son los básicos, seguido de los medicamentos y electrodométsicos

In [10]:
# table count of each catalog, show only first 20
table_count_catalogue = df_qqp.groupBy('year', 'catalog').count().orderBy('year', 'catalog')

# get only first 20 per year
table_count_catalogue = (
    table_count_catalogue
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('year').orderBy(F.desc('count'))
        ))
    .filter('rank <= 20')
    )
table_count_catalogue.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------------------+-------+----+
|year|            catalog|  count|rank|
+----+-------------------+-------+----+
|2018|            basicos|8804761|   1|
|2018|       medicamentos|3657969|   2|
|2018|  electrodomesticos|1735336|   3|
|2018| frutas y legumbres| 938616|   4|
|2018|   utiles escolares| 665135|   5|
|2018|           mercados| 457497|   6|
|2018|           juguetes| 370421|   7|
|2018|pescados y mariscos|  90040|   8|
|2018|          navidenos|  58674|   9|
|2018|              tenis|   4452|  10|
|2019|            basicos|7398860|   1|
|2019|       medicamentos|2887105|   2|
|2019|  electrodomesticos|1352068|   3|
|2019| frutas y legumbres| 702747|   4|
|2019|   utiles escolares| 575317|   5|
|2019|           mercados| 381183|   6|
|2019|           juguetes| 267730|   7|
|2019|pescados y mariscos|  69658|   8|
|2019|          navidenos|  42538|   9|
|2019|              tenis|   4306|  10|
+----+-------------------+-------+----+
only showing top 20 rows

En una tabla aparte se pueden observar los 20 catalogos con más productos en cada año

### Q3: ¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [11]:
# get table count for each year catalog the # of distinct states
table_count_num_states_per_year_catalog = (
    df_qqp
    .groupBy('catalog', 'state', 'year')
    .count()
    .orderBy('catalog', 'state', 'year')
    )

# generate table of # of distinct states per year and catalog
distinct_states = df_qqp.select('state').distinct()
distinct_years = df_qqp.select('year').distinct()
distinct_catalogs = df_qqp.select('catalog').distinct()

df_cross_join_state_year_catalog = (
    distinct_catalogs
    .crossJoin(distinct_states)
    .crossJoin(distinct_years)
    )

# get which catalog state and year are missing
table_missing = (
    df_cross_join_state_year_catalog
    .join(table_count_num_states_per_year_catalog, ['catalog', 'state', 'year'], 'left')
    .filter('count is null')
    )

# show missing
print(f"Missing: {table_missing.count()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Missing: 586

Hay 586 combinaciones entre catálogo, estado y año que no tienen información. En una tabla aparte se pueden visualizar cuales son.

In [12]:
# look
table_missing.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+----+-----+
|           catalog|              state|year|count|
+------------------+-------------------+----+-----+
|             pacic|           guerrero|2021| NULL|
|          juguetes|            nayarit|2024| NULL|
|             pacic|       quintana roo|2021| NULL|
|  utiles escolares|            sinaloa|2023| NULL|
|             pacic|          queretaro|2021| NULL|
|             tenis|           tlaxcala|2024| NULL|
|           basicos|            nayarit|2024| NULL|
|             pacic|            durango|2019| NULL|
|  utiles escolares|           guerrero|2023| NULL|
|       aeropuertos|          queretaro|2023| NULL|
|frutas y legumbres|            nayarit|2020| NULL|
|  utiles escolares|             sonora|2020| NULL|
|       aeropuertos|baja california sur|2019| NULL|
|             pacic|          chihuahua|2021| NULL|
|             tenis|    baja california|2024| NULL|
|             tenis|            yucatan|2024| NULL|
|       aero

En una tabla aparte se puede observar la tabla con los catalogos vacios por año y estado

In [13]:
# count the # of rows per state and year
table_count_states = df_qqp.groupBy('state', 'year').count().orderBy('state', 'year')

# pivot the table to have years as columns
(
    table_count_states
    .groupBy('state')
    .pivot('year')
    .agg(F.coalesce(F.lit(1), F.lit(0)))
    .orderBy('state')
    .show(32)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----+----+----+----+----+----+
|               state|2018|2019|2020|2021|2022|2023|2024|
+--------------------+----+----+----+----+----+----+----+
|      aguascalientes|   1|   1|   1|   1|   1|   1|   1|
|     baja california|   1|   1|   1|   1|   1|   1|   1|
| baja california sur|   1|   1|   1|   1|   1|   1|   1|
|            campeche|   1|   1|   1|   1|   1|   1|   1|
|             chiapas|   1|   1|   1|   1|   1|   1|   1|
|           chihuahua|   1|   1|   1|   1|   1|   1|   1|
|    ciudad de mexico|   1|   1|   1|   1|   1|   1|   1|
|coahuila de zaragoza|   1|   1|   1|   1|   1|   1|   1|
|              colima|   1|   1|   1|NULL|NULL|NULL|NULL|
|             durango|   1|   1|   1|   1|   1|   1|   1|
|    estado de mexico|   1|   1|   1|   1|   1|   1|   1|
|          guanajuato|   1|   1|   1|   1|   1|   1|   1|
|            guerrero|   1|   1|   1|   1|   1|   1|   1|
|             hidalgo|   1|   1|   1|   1|   1|   1|   1|
|             

Los estados que no tienen información de ciertos catálogos son los estados de Colima y Nayarit a partir del 2020

### Q4: ¿Cuántas observaciones tenemos por estado?


In [14]:
# count the # of rows per state
df_qqp.groupBy('state').count().orderBy('state').show(32)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|               state|   count|
+--------------------+--------+
|      aguascalientes| 1668545|
|     baja california| 1513987|
| baja california sur| 1477127|
|            campeche| 1765074|
|             chiapas| 1163732|
|           chihuahua| 1798478|
|    ciudad de mexico|17968798|
|coahuila de zaragoza| 1616519|
|              colima|  560520|
|             durango| 1616227|
|    estado de mexico|12537548|
|          guanajuato| 3330005|
|            guerrero|  981815|
|             hidalgo|  868532|
|             jalisco| 4182990|
| michoacan de ocampo| 1538864|
|             morelos| 1026711|
|             nayarit|  266068|
|          nuevo leon| 3358320|
|              oaxaca| 1536513|
|              puebla| 2345346|
|           queretaro| 2534860|
|        quintana roo| 2821004|
|     san luis potosi| 1718257|
|             sinaloa| 1281809|
|              sonora| 1887619|
|             tabasco| 2471506|
|          tamaulipas| 1896217|
|       

La tabla superior muestra los conteos de observaciones por estado. La `ciudad de méxico` es la que predomina

In [15]:
# count the number of different catalogs (WIP)
table_qqp_state_catalog_year = (
    df_qqp
    .groupBy('state', 'catalog', 'year')
    .count()
    .orderBy('state', 'catalog', 'year')
    )
table_qqp_state_catalog_year.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------+----+------+
|         state|           catalog|year| count|
+--------------+------------------+----+------+
|aguascalientes|       aeropuertos|2019|   121|
|aguascalientes|           basicos|2018|106821|
|aguascalientes|           basicos|2019|137394|
|aguascalientes|           basicos|2020|105656|
|aguascalientes|           basicos|2021|112011|
|aguascalientes|           basicos|2022|136357|
|aguascalientes|           basicos|2023|137803|
|aguascalientes|           basicos|2024| 37042|
|aguascalientes| electrodomesticos|2018| 26044|
|aguascalientes| electrodomesticos|2019| 37355|
|aguascalientes| electrodomesticos|2020| 18383|
|aguascalientes| electrodomesticos|2021| 26257|
|aguascalientes| electrodomesticos|2022| 31491|
|aguascalientes| electrodomesticos|2023| 36656|
|aguascalientes| electrodomesticos|2024|  8228|
|aguascalientes|frutas y legumbres|2018| 12824|
|aguascalientes|frutas y legumbres|2019| 14427|
|aguascalientes|frutas y legumbres|2020|

En una tabla por aparte se pueden observar el numero de observaciones por estado, catálogo y año

### Q5: De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [16]:
# count the number of different catalogs per year
table_qqp_year_state_diff_catalog = (
    df_qqp
    .groupBy('state', 'year')
    .agg(F.countDistinct('catalog').alias('count_catalogs'))
    .orderBy('state', 'year')
    .groupBy('state')
    .pivot('year')
    .agg(F.first('count_catalogs'))
    .orderBy('state')
    )
table_qqp_year_state_diff_catalog.show(32)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----+----+----+----+----+----+
|               state|2018|2019|2020|2021|2022|2023|2024|
+--------------------+----+----+----+----+----+----+----+
|      aguascalientes|   9|  11|   9|  10|  11|  11|   9|
|     baja california|  10|  10|   9|   9|  11|  11|  10|
| baja california sur|   9|  10|   9|  10|  11|  11|  10|
|            campeche|   9|   9|   9|  10|  11|  11|   9|
|             chiapas|  10|  10|   9|   9|  10|  10|   8|
|           chihuahua|  10|  11|   5|  10|  11|  11|  10|
|    ciudad de mexico|  10|  10|   9|  10|  11|  11|  10|
|coahuila de zaragoza|  10|   9|   9|  10|  11|  11|  10|
|              colima|  10|  10|   4|NULL|NULL|NULL|NULL|
|             durango|  10|  10|   9|  10|  11|  11|   9|
|    estado de mexico|  10|  10|   9|  10|  11|  11|  10|
|          guanajuato|  10|  10|   9|  10|  11|  11|   9|
|            guerrero|  10|  10|   8|   8|   8|   7|   9|
|             hidalgo|  10|  11|   9|   9|  10|  11|   8|
|             

Falta observar tendencias en el tiempo para afirmar si hay un crecimiento en el número de catálogos por estado, sin embargo se muestra como para Colima y Nayarit bajó

### Save tables

In [17]:
# save tables to s3
table_count_catalogue.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_count_catalogue")
table_missing.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_missing")
table_qqp_state_catalog_year.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_qqp_state_catalog_year")
table_qqp_year_state_diff_catalog.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_qqp_year_state_diff_catalog")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# look schema
df_qqp.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product: string (nullable = true)
 |-- presentation: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- created_at: date (nullable = true)
 |-- store: string (nullable = true)
 |-- type_of_store: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- catalog: string (nullable = true)
 |-- year: integer (nullable = true)

In [ ]:
# save transformed data to s3
df_qqp.write.mode('overwrite').partitionBy('catalog').parquet(f"{BUCKET}/{FOLDER}/{NAME_FILE}_transformed")
df_qqp.write.mode('overwrite').partitionBy('catalog').parquet(f"{BUCKET}/{FOLDER}/{NAME_FILE}_transformed_unpartitioned")

## S2: Particular

### Get Data

In [19]:
# filter by catalog
df_qqp_our_category = df_qqp.filter(col('catalog') == CATALOG_TYPE)

# look # of rows
df_qqp_our_category.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19223453

### Q1: ¿Cuańtas marcas diferentes tiene tu categoría?

In [20]:
# how many different brands are there?
df_qqp_our_category.select('brand').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

40

### Q2: ¿Cuál es la marca con mayor precio? ¿En qué estado?

In [21]:
# get avg by brand and state
df_qqp_our_category_avg_price_per_brand_state = (
    df_qqp_our_category
    .groupBy('brand', 'state')
    .agg(F.avg('price').alias('avg_price'))
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# which brand has the highest price and in which state?
(
    df_qqp_our_category_avg_price_per_brand_state
    .orderBy(col('avg_price').desc())
    .select('brand', 'state', 'avg_price').show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+---------+
| brand|   state|avg_price|
+------+--------+---------+
|jumper|tlaxcala| 1381.625|
+------+--------+---------+
only showing top 1 row

Hay un producto `sin marca` en el estado de `Chihuaha` con un precio de $5,275; el cual sería el máximo

In [33]:
# which brand has the highest price and in which state?
(
    df_qqp_our_category_avg_price_per_brand_state
    .filter(~col('brand').isin('s/m'))
    .orderBy(col('avg_price').desc())
    .select('brand', 'state', 'avg_price').show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+---------+
| brand|   state|avg_price|
+------+--------+---------+
|jumper|tlaxcala| 1381.625|
+------+--------+---------+
only showing top 1 row

El producto `omron` es el producto _identificable_ más caro de QQP, procedente del estadio de `Baja California Sur`

### Q3: ¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [24]:
# idem but in CDMX
(
    df_qqp_our_category_avg_price_per_brand_state
    .filter(col('state') == 'ciudad de mexico')
    .orderBy(col('avg_price').desc())
    .select('brand', 'state', 'avg_price').show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+------------------+
|     brand|           state|         avg_price|
+----------+----------------+------------------+
|choisemmed|ciudad de mexico|1001.5555320366128|
+----------+----------------+------------------+
only showing top 1 row

El producuto más caro es de $4521, `sin marca`

In [36]:
# idem CDMX without s/m
(
    df_qqp_our_category
    .filter(~col('brand').isin('s/m'))
    .filter(col('state') == 'ciudad de mexico')
    .groupBy('brand')
    .agg(F.avg('price').alias('avg_price'))
    .orderBy(col('avg_price').desc())
    .select('brand', 'avg_price')
    .show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+
|     brand|         avg_price|
+----------+------------------+
|choisemmed|1001.5555320366128|
+----------+------------------+
only showing top 1 row

El producto `omron` es el producto _identificable_ más caro de la CDMX

### Q4: ¿Cuál es la marca con mayores observaciones?

In [26]:
# which brand has more observations?
table_brand_count = df_qqp_our_category.groupBy('brand').count().orderBy('count', ascending=False)
table_brand_count.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|               brand|   count|
+--------------------+--------+
|                 s/m|17840684|
|           sin marca|  236545|
|     cualquier marca|  189801|
|              le roy|  164858|
|                zuum|  105398|
|               omron|   80857|
|            medimart|   70602|
|               dalux|   68836|
|              protec|   62126|
|              ensure|   49280|
|         quality day|   39434|
|             nebucor|   31963|
|escudo antibacterial|   31002|
|            vitascom|   30293|
|             curitas|   29333|
|             soriana|   26033|
|inhala care  o ho...|   21639|
|          zuum. klin|   18977|
|           microlife|   18252|
|         3m. nexcare|   15338|
+--------------------+--------+
only showing top 20 rows

La marca _identificable_ con más observaciones es la marca `le roy`

### Q5: ¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

In [27]:
# for each state, get the top 5 most expensive brands (WIP)
table_price_brand_top5 = (
    df_qqp_our_category_avg_price_per_brand_state
    .orderBy(col('avg_price').desc())
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('state').orderBy(F.desc('avg_price'))
        ))
    .filter('rank <= 5')
    # pivot brand and set values the rank
    .groupBy('state')
    .pivot('brand')
    .agg(F.first('rank'))
    .orderBy('state')
    )
table_price_brand_top5.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------+----------+-------+-------+------+---------+--------------------+------------------------+------+---------+-------+-----+------+----+------+
|              state|benesta|choisemmed|citizen|duo max|ensure|home care|home care diagnostic|inhala care  o home care|jumper|microlife|nebucor|omron|protec| s/m|xignal|
+-------------------+-------+----------+-------+-------+------+---------+--------------------+------------------------+------+---------+-------+-----+------+----+------+
|     aguascalientes|   NULL|         1|   NULL|   NULL|  NULL|     NULL|                   2|                    NULL|  NULL|        5|      3|    4|  NULL|NULL|  NULL|
|    baja california|   NULL|         2|   NULL|   NULL|  NULL|     NULL|                   1|                    NULL|     5|     NULL|      4| NULL|  NULL|NULL|     3|
|baja california sur|   NULL|         2|   NULL|   NULL|  NULL|     NULL|                   1|                    NULL|     4|     NULL|      5|    3|

Las top 5 marcas con mayo precio se pueden encontrar en una tabla aparte

### Q6: ¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [28]:
# top 5 least expensive brands in CDMX
(
    df_qqp_our_category_avg_price_per_brand_state
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('state').orderBy('avg_price')
        ))
    .filter(col('state') == 'ciudad de mexico')
    .filter('rank <= 5')
    .show(6)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+------------------+----+
|               brand|           state|         avg_price|rank|
+--------------------+----------------+------------------+----+
|            dr. simi|ciudad de mexico|              14.0|   1|
|            curapack|ciudad de mexico|20.723846153846157|   2|
|escudo antibacterial|ciudad de mexico|21.511706203035242|   3|
|          zuum. klin|ciudad de mexico|21.602950886488625|   4|
|               hands|ciudad de mexico| 22.16253561253561|   5|
+--------------------+----------------+------------------+----+

Las marcas _identificables_ con el menor precio de la CDMX son `protec` `quality day` `soriana` `farmacom` y `escudo antibacterial`

### Q7: ¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

In [29]:
# top 5 brands with more observations per state (WIP)
(
    df_qqp_our_category
    .groupBy('brand')
    .count()
    .orderBy('count', ascending=False)
    .show(6)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------+
|          brand|   count|
+---------------+--------+
|            s/m|17840684|
|      sin marca|  236545|
|cualquier marca|  189801|
|         le roy|  164858|
|           zuum|  105398|
|          omron|   80857|
+---------------+--------+
only showing top 6 rows

Las marcas _identificables_ con el más productos del pais son `le roy` `zuum` `omron` `medimart` y `dalux`

In [30]:
# table of counts by state and brand and get top 5 brands per state
table_brand_state_count_top5 = (
    df_qqp_our_category
    .groupBy('state', 'brand')
    .count()
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('state').orderBy(F.desc('count'))
        ))
    .filter('rank <= 5')
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

En una tabla aparte se puede observar las 5 marcas con más observaciones por estado

### Q8: ¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

In [31]:
# distinct observeations for year and brand (WIP)
table_brand_year_count = (
    df_qqp_our_category
    .groupBy('year', 'brand')
    .count()
    .orderBy('year', 'count')
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

En una tabla aparte se puede observar la tabla con las marcas que han dejado de existir y en el año en la que dejó de existir

### Q9: Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.

In [32]:
# for each year, get the most expensive brand (WIP)
table_state_year_max_price = (
    df_qqp_our_category
    # get avg price per brand, state and year
    .groupBy('year', 'brand', 'state')
    .agg(F.avg('price').alias('avg_price'))
    # get the max price per state and year
    .groupBy('year', 'state')
    .agg(F.max('avg_price').alias('max_price'))
    .orderBy('year', 'state')
    
    )
table_state_year_max_price.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+------------------+
|year|               state|         max_price|
+----+--------------------+------------------+
|2018|      aguascalientes|388.22660206105945|
|2018|     baja california| 419.5214806359877|
|2018| baja california sur|402.06128286014683|
|2018|            campeche|381.20061637415125|
|2018|             chiapas| 403.2377926083793|
|2018|           chihuahua|448.99039110831933|
|2018|    ciudad de mexico|440.91772056453067|
|2018|coahuila de zaragoza| 403.1310508026669|
|2018|              colima| 387.3165168519595|
|2018|             durango| 396.9891012485812|
|2018|    estado de mexico| 436.0948229431829|
|2018|          guanajuato| 458.9055676704322|
|2018|            guerrero|  392.859522975657|
|2018|             hidalgo|   440.05489464648|
|2018|             jalisco|396.75492776932094|
|2018| michoacan de ocampo|462.06874465090885|
|2018|             morelos| 408.8609457837893|
|2018|             nayarit|420.89165703734733|
|2018|       

En un gráfico por aparte se puede ver la tendencia de los precios máximos por estado en cada año

## Save

In [37]:
# sabe tables to s3
table_brand_count.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_count")
table_price_brand_top5.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/price_brand_top5")
table_brand_year_count.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_state_count")
table_brand_state_count_top5.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_year_count_top5")
table_state_year_max_price.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/state_year_max_price")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…